In [ ]:
from IPython.display import Audio
import soundfile as sf
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
import os
import librosa
import librosa.display
from tqdm import tqdm_notebook
from pathlib import Path
import shutil
import numpy as np
import scipy
from random import shuffle

In [ ]:
import json
with open('metadata_dict.json','r') as f:
    metadata_dict = json.load(f)

In [ ]:
dataset = 'audio_biggerthan2sec'

for split in ['train','dev','eval']:
    for label in ['real','fake']:
        Path(f'{dataset}/{split}/{label}').mkdir(parents=True, exist_ok = True)

for split in ['train','dev','eval']:
    for label in ['real']:
        files = metadata_dict[f'{split}_{label}']
        for file in tqdm_notebook(files, desc = f'{split}_{label}'):
            clip, sample_rate = librosa.load(f'LA/ASVspoof2019_LA_{split}/flac/{file}.flac', sr=None)
            if len(clip) >= sample_rate*2:
                shutil.copy(f'LA/ASVspoof2019_LA_{split}/flac/{file}.flac',f'{dataset}/{split}/{label}/{file}.flac')

for split in ['train','dev','eval']:
    for label in ['fake']:
        files = metadata_dict[f'{split}_{label}']
        count = 0
        num_required = len(os.listdir(f'{dataset}/{split}/real'))
        for file in tqdm_notebook(files, desc = f'{split}_{label}'):
            if count == num_required:
                break
            clip, sample_rate = librosa.load(f'LA/ASVspoof2019_LA_{split}/flac/{file}.flac', sr=None)
            if len(clip) >= sample_rate*2:
                shutil.copy(f'LA/ASVspoof2019_LA_{split}/flac/{file}.flac',f'{dataset}/{split}/{label}/{file}.flac')
                count += 1

In [ ]:
dataset = 'audio_biggerthan2sec'
clip_length = 32000 # As samplerate is 16 KHz
new_dataset = f'npys_{clip_length}'
for split in ['train','dev','eval']:
    for label in ['real','fake']:
        Path(f'{new_dataset}/{split}/{label}').mkdir(parents=True, exist_ok=True)
        files = os.listdir(f'{dataset}/{split}/{label}')
        for file in tqdm_notebook(files):
            clip, sample_rate = librosa.load(f'{dataset}/{split}/{label}/{file}', sr=None)
            arr = clip[:clip_length]
            filename = file.split('.')[0]
            np.save(f'{new_dataset}/{split}/{label}/{filename}.npy', arr)

In [ ]:
for split in ['train','dev','eval']:
    for label in ['real','fake']:
        files = os.listdir(f'{dataset}/{split}/{label}')
        for file in tqdm_notebook(files):
            clip, sample_rate = librosa.load(f'{dataset}/{split}/{label}/{file}', sr=None)
            assert len(clip)/sample_rate >= 2

In [ ]:
for split in ['train','dev','eval']:
    for label in ['real','fake']:
        length = len(os.listdir(f'{dataset}/{split}/{label}'))
        print(f'{dataset}/{split}/{label}: {length}')